In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dir_path = '/content/drive/My Drive/Colab Notebooks'
import os
os.chdir(dir_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


70

In [ ]:
import gensim
import re
import torch
import numpy as np

w2v_path = 'GoogleNews-vectors-negative300.bin (1).gz'
data_path = 'report8/data70/'

w2v_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_path , binary=True)

category2num = {"b": 0, "t": 1, "e": 2, "m": 3}

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# タイトルを受け取り，単語ベクトルの平均を返す関数
def get_feature(title):
    word_list = title.split(' ')#todo タイトルをスペースで分割
    vec_list = []
    for word in word_list:
        try:
            vec = w2v_model[word]#todo wordを意味するベクトルを取得
        except KeyError:
            vec = np.zeros(300)#todo すべての要素が0のベクトルを代入
        vec_list.append(vec)    
    vec_np = np.array(vec_list) # numpyのarrayに変換
    feature = np.sum(vec_np)/len(vec_np)# 平均ベクトルを計算
    return feature


def get_data(fname):
    label_list = []
    feature_list = []
    with open(fname, encoding="utf8") as f:
        for line in f:
            if not line:
                continue
            data = line.split('\t')
            title = data[1]#todo dataからタイトルを取り出す
            feature = get_feature(title) 
            feature_list.append(feature)
            label = category2num[data[0]]
            label_list.append(label)

    features = torch.tensor(feature_list)#todo feature_listをtensorに変換
    labels = torch.tensor(label_list)#todo label_listをtensorに変換
    return features, labels


train_x, train_y = get_data(data_path + "train.txt")
valid_x, valid_y = get_data(data_path + "valid.txt")
test_x, test_y = get_data(data_path + "test.txt")

# 保存
torch.save(train_x, data_path + "train_x.pt")
torch.save(train_y, data_path + "train_y.pt")
torch.save(valid_x, data_path + "valid_x.pt")
torch.save(valid_y, data_path + "valid_y.pt")
torch.save(test_x, data_path + "test_x.pt")
torch.save(test_y, data_path + "test_y.pt")
